In [ ]:
!nvidia-smi
from google.colab import drive
drive.mount('/content/drive')


Tue Feb 25 05:05:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             67W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
!pip install transformers==4.33.0 torchmetrics
!pip install git+https://github.com/haotian-liu/LLaVA.git@v1.6
!pip install git+https://github.com/salesforce/LAVIS.git

import os, json, random
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoProcessor, Blip2ForConditionalGeneration
from torchmetrics.classification import Accuracy
from difflib import SequenceMatcher
from PIL import Image


  Cloning https://github.com/haotian-liu/LLaVA.git (to revision v1.6) to /tmp/pip-req-build-toi6x0wo
  Running command git clone --filter=blob:none --quiet https://github.com/haotian-liu/LLaVA.git /tmp/pip-req-build-toi6x0wo
  Running command git checkout -q v1.6
  error: pathspec 'v1.6' did not match any file(s) known to git
  error: subprocess-exited-with-error
  
  × git checkout -q v1.6 did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git checkout -q v1.6 did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
  Cloning https://github.com/salesforce/LAVIS.git to /tmp/pip-req-build-6d0at52v
  Running command git clone --filter=blob:none --quiet https://github.com/salesforce/LAVIS.git /tmp/pip-req-build-6d0at52v
  Resolved https://github.co

/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [ ]:
PROJECT_DIR = "/content/drive/MyDrive/mmml_project_michelle/mmml_project"
os.chdir(PROJECT_DIR)

mini_gqa_path = "mini_gqa.json"
with open(mini_gqa_path, 'r') as f:
    mini_gqa_data = json.load(f)

print(f"Loaded {len(mini_gqa_data)} samples from mini_gqa.json.")
print("Example sample:", mini_gqa_data[0])


Loaded 12578 samples from mini_gqa.json.
Example sample: {'imageId': 'n161313', 'image_file': '/content/drive/MyDrive/mmml_project/gqa_images/n161313.png', 'question': 'Is it overcast?', 'answer': 'no', 'fullAnswer': 'No, it is clear.'}


In [ ]:
old_prefix = "/content/drive/MyDrive/mmml_project/gqa_images/"
new_prefix = os.path.join(PROJECT_DIR, "gqa_images") + "/"
for sample in mini_gqa_data:
    if sample.get("image_file", "").startswith(old_prefix):
        sample["image_file"] = sample["image_file"].replace(old_prefix, new_prefix)
print("Updated sample record:", mini_gqa_data[0])

Updated sample record: {'imageId': 'n161313', 'image_file': '/content/drive/MyDrive/mmml_project_michelle/mmml_project/gqa_images/n161313.png', 'question': 'Is it overcast?', 'answer': 'no', 'fullAnswer': 'No, it is clear.'}


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

def similarity(a: str, b: str) -> float:
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

def is_correct(prediction: str, gold: str, threshold=0.8) -> bool:
    return similarity(prediction, gold) >= threshold


Using device: cuda


In [ ]:
import time

from transformers import AutoProcessor, Blip2ForConditionalGeneration

processor_weak = AutoProcessor.from_pretrained("Salesforce/blip2-flan-t5-xl", use_fast=False)
model_weak = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xl").to(device)

weak_predictions_list = []

save_interval = 500
start_time = time.time()

total_samples = len(mini_gqa_data)
for i, sample in enumerate(mini_gqa_data):
    qid = sample["imageId"]
    img_path = sample["image_file"]
    question = sample["question"]

    try:
        image = Image.open(img_path).convert("RGB")
    except Exception as e:
        print(f"Error opening {img_path}: {e}")
        continue

    inputs = processor_weak(image, question, return_tensors="pt").to(device)
    with torch.no_grad():
        generated_ids = model_weak.generate(**inputs, max_new_tokens=20)
    pred_answer = processor_weak.tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    entry = {
        "imageId": sample["imageId"],
        "image_file": sample["image_file"],
        "question": sample["question"],
        "answer": sample["answer"],
        "fullAnswer": sample.get("fullAnswer", ""),
        "blip_response": pred_answer
    }

    weak_predictions_list.append(entry)

    if (i + 1) % save_interval == 0:
        elapsed = time.time() - start_time
        print(f"Processed {i+1}/{total_samples} samples. Time elapsed: {elapsed/60:.2f} minutes.")
        temp_path = os.path.join(PROJECT_DIR, "mini_gqa_with_weakB_fulljson_checkpoint.json")
        with open(temp_path, "w") as f:
            json.dump(weak_predictions_list, f, indent=2)

weak_out_path = os.path.join(PROJECT_DIR, "mini_gqa_with_weakB_fulljson.json")
with open(weak_out_path, "w") as f:
    json.dump(weak_predictions_list, f, indent=2)

print(f"WeakB predictions saved to: {weak_out_path}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (pr

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Processed 500/12578 samples. Time elapsed: 1.68 minutes.
Processed 1000/12578 samples. Time elapsed: 3.35 minutes.
Processed 1500/12578 samples. Time elapsed: 5.05 minutes.
Processed 2000/12578 samples. Time elapsed: 6.73 minutes.
Processed 2500/12578 samples. Time elapsed: 8.44 minutes.
Processed 3000/12578 samples. Time elapsed: 10.13 minutes.
Processed 3500/12578 samples. Time elapsed: 11.86 minutes.
Processed 4000/12578 samples. Time elapsed: 13.53 minutes.
Processed 4500/12578 samples. Time elapsed: 15.22 minutes.
Processed 5000/12578 samples. Time elapsed: 16.94 minutes.
Processed 5500/12578 samples. Time elapsed: 18.64 minutes.
Processed 6000/12578 samples. Time elapsed: 20.31 minutes.
Processed 6500/12578 samples. Time elapsed: 22.01 minutes.
Processed 7000/12578 samples. Time elapsed: 23.70 minutes.
Processed 7500/12578 samples. Time elapsed: 25.42 minutes.
Processed 8000/12578 samples. Time elapsed: 27.14 minutes.
Processed 8500/12578 samples. Time elapsed: 28.82 minutes.
Pro